In [1]:
import os
import s3fs
os.environ["AWS_ACCESS_KEY_ID"] = 'BFMNK1LUPXQ061C6PZ55'
os.environ["AWS_SECRET_ACCESS_KEY"] = '1lUWhLj95YhcD8P5t6GEMXxwlf0jyzmqrGPMURQE'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJCRk1OSzFMVVBYUTA2MUM2UFo1NSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzU5MDQ2Njc3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzU5NjUxNTE1LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc1OTA0NjcxNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDplNzkxZWViNS1kYWQyLWIxZjYtMzJjZi05YTM2ODVjNTNlNGIiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImFmY2VhMjNmLTM3MGUtNDhjNC1hNGQ5LWIzZDJjMDJkYmNkOCIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.JF4l6bNSZ2jJniDyrrDJRipM1GFJh5a05vfMLSwOcL5vXJbg6BjsZ639IP1NSk2phHnYPPnpD8xnCk6Pub1mEw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"


In [2]:
%pip install lxml
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd


articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="article", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors_str
            })

            elem.clear()

df = pd.DataFrame(articles)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.4 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
df

,title,journal,year,authors
0,Auswirkung der Digitalisierung auf die Systeml...,Elektrotech. Informationstechnik,2019,Kai Schlabitz
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,Richard berbacher; Stefan Cecil
2,Zur Genesis der Forschungsstelle fr Integriert...,Elektrotech. Informationstechnik,2022,Herbert Mang
3,100 % erneuerbare Energie fr sterreichs Indust...,Elektrotech. Informationstechnik,2021,Sophie Knttner; Roman Geyer; Christian Diendor...
4,Application of multilateration for microphone ...,Elektrotech. Informationstechnik,2021,Peter Wimberger; Christoph Reichl
...,...,...,...,...
3992218,"Derivability, Redundancy and Consistency of Re...","Research Report / RJ / IBM / San Jose, California",1969,E. F. Codd
3992219,Normalized Data Base Structure: A Brief Tutorial.,"Research Report / RJ / IBM / San Jose, California",1971,E. F. Codd
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,Markus Casper; Gayane Grigoryan; Oliver Gronz;...
3992221,Common Subexpression Identification in General...,"Technical Rep. UKSC 0060, IBM United Kingdom S...",1974,Patrick A. V. Hall


In [5]:
chunks = []
n_chunks = 50
chunk_size = len(df) // n_chunks

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(df)  # last chunk includes remainder
    
    chunk = df.iloc[start:end].copy()
    
    # Split authors column into multiple columns
    #authors_df = chunk['authors'].str.split(';', expand=True)
    #authors_df.columns = [f'author{i+1}' for i in range(authors_df.shape[1])]
    
    # Combine with original chunk
    #chunk = pd.concat([chunk.drop(columns=['authors']), authors_df], axis=1)
    
    chunk.to_parquet(f"t/articles_chunk_{i}.parquet", index=False)

In [13]:

import pandas as pd
from collections import defaultdict
import glob

ego_data = defaultdict(lambda: {"publications": []}) #set permet de créer des clefs si absentes et de les actualiser si présentes (en définissant la classe des clefs)

chunk_files = glob.glob("t/articles_chunk_*.parquet")

for file in chunk_files:
    df_chunk = pd.read_parquet(file)
    """
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        for ego in authors:
            coauthors = set(authors) - {ego}
            #ego_data[ego]["coauthors"].update(coauthors)
            
            ego_data[ego]["covariates"].update({
                "coauthors" : list(coauthors),
                "journal": row.get("journal"),
                "year": row.get("year"),
                "title" : row.get("title")
            })
    """
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        for ego in authors:
            coauthors = set(authors) - {ego}
            
            ego_data[ego]["publications"].append({
                "coauthors": list(coauthors),
                "journal": row.get("journal"),
                "year": row.get("year"),
                "title": row.get("title")
            })

In [15]:
"""ego_df = pd.DataFrame([
    {'ego': ego, **data["publications"]} #récup paries ego - alter et trie dans l'ordre alphabétique
    for ego, data in ego_data.items()
])"""
rows = []
for ego, data in ego_data.items():
    for pub in data["publications"]:
        rows.append({
            "ego": ego,
            "coauthors": ",".join(sorted(pub["coauthors"])),
            "journal": pub.get("journal"),
            "year": pub.get("year"),
            "title": pub.get("title")
        })

ego_df = pd.DataFrame(rows)


In [23]:
ego_df["ego"].str.contains("Jing Huang 0007").sum()

np.int64(67)

In [ ]:
ego_df.to_parquet("ego_coauthors.parquet", index=False)
s3_path = "s3://lab/ego_coauthors.parquet"  # bucket + key

# Save directly to MinIO
ego_df.to_parquet(s3_path, engine="pyarrow", index=False, filesystem=fs)

In [8]:
ego_df.to_parquet("ego_df.parquet",index=False)
